1 定义工具函数

配置好环境变量后，我们就可以通过Assistant API来构建一个Agent了。
首先我们需要定义一些工具函数，假设我们的答疑机器人有三个功能，查询员工信息，发送请假申请，查询公司规章制度。
我们需要定义三个工具函数。

1.1 查询员工信息

你希望将用户的提问作为工具函数的输入，并生成对应的SQL语句（这一步骤也叫NL2SQL，即自然语言转SQL语句），再使用SQL语句去数据库中查询，将查询结果作为工具函数的输出。为了帮助你更多的关注到Agent的内容，我们模拟一个查询步骤，而不实际去数据库中查询。

假设员工表名为employee，字段包括department（部门）、name（姓名）、HR。

In [ ]:
from llama_index.llms.dashscope import DashScope
from llama_index.core.base.llms.types import MessageRole, ChatMessage

def query_employee_info(query):
  llm = DashScope(model_name="qwen-plus")
  messages = [
    ChatMessage(role=MessageRole.SYSTEM, content='''你有一个表叫employees，记录公司的员工信息，这个表有department（部门）、name（姓名）、HR三个字段。
      你需要根据用户输入生成sql语句进行查询,你一定不能生成sql语句之外的内容，也不要把```sql```这个信息加上。
    '''),
    ChatMessage(role=MessageRole.USER, content=query)
  ]
  SQL_output = llm.chat(messages).message.content.lower()
  print(f'SQL statement is: {SQL_output}')
  # 2. 根据SQL语句去查询数据库（此处为模拟查询），并返回结果
  if SQL_output == "select count(*) from employees where department = '教育部门'":
      return "教育部门共有66名员工。"
  if SQL_output == "SELECT HR FROM employees WHERE name = '张三'".lower():
      return "张三的HR是李四。"
  if SQL_output == "select department from employees where name = '王五'".lower():
      return "王五的部门是后勤部。"
  else:
      return "抱歉，我暂时无法回答您的问题。"

# query_employee_info("王五在哪个部门？")
# print("*"*12)
query_employee_info("教育部门有几个人")

************
SQL statement is: select count(*) from employees where department = '教育部门'


'教育部门共有66名员工。'

1.2 发送请假申请

你希望这个工具函数将员工输入的请假日期作为输入参数，并返回一个申请成功的字符串。我们在此模拟一个申请步骤，而不实际去公司系统中提交请假申请。

In [23]:
def send_leave_application(date):
  '''
  输入请假时间，输出请假申请发送结果
  '''
  return f'已为你发送请假申请，请假日期是{date}'

# 测试一下这个函数
print(send_leave_application("明后两天"))

已为你发送请假申请，请假日期是明后两天


1.3 查询公司规章制度

你希望这个工具函数以用户的提问作为输入，输出是根据之前教程创建的RAG答疑机器人给出的回复。

In [ ]:
import sys
sys.path.append("..")
from chatbot.rag import load_index

def query_company_info(query):
  '''
  输入用户提问，输出公司信息查询结果
  '''
  # 使用封装好的函数加载索引
  index = load_index()
  query_engine = index.as_query_engine(
    llm=DashScope(model_name="qwen-plus")
  )
  return query_engine.query(query).response
# 测试一下效果
query_company_info("我们公司项目管理应该用什么工具")

尽管这些工具函数的实现比较简单，但却是大模型与外界交互的关键。接下来我们将使用Assistant API创建一个Agent，并将这三个工具函数集成进来。

2 将工具函数与大模型集成进Agent中

你已经定义好了工具函数，接下来就要将它们与大模型通过Assistant API集成到Agent中。

通过Assistants.create方法，你可以创建一个新的Agent，并通过model（模型名称）、name（Agent命名）、description（Agent的描述信息）、instructions（Agent非常重要的参数，用于提示Agent所具有的工具函数能力，同时也可以规范输出格式）、tools（工具函数通过该参数传入）参数来定义Agent。

其中，tools参数中的function.name用于指定工具函数，但需要为字符串格式，因此可以通过一个map方法映射到工具函数上。

通过Assistant API获得Agent回复的过程需要涉及到如thread、message、run等概念，如果你对这些概念与细节感兴趣，请参考阿里云Assistant API官方文档。

我们封装好了一个辅助函数get_agent_response，你只需要传入Agent与提问内容，即可得到指定Agent的回复。

如果你希望给Agent配备更多的能力，可以添加工具函数，并在function_mapper与tools中建立映射关系。

我们可以向ChatAssistant提问，测试一下回复效果。

从测试结果可以看出，拓展了功能之后的答疑机器人达到了你预期的效果。